# Self-Organising Maps


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch, Circle, Rectangle


# Cargar dataset de entrenamiento
datos = pd.read_csv("pokemon_train.csv")
# Seleccionar columnas "against_" (18 dimensiones)
columnas_against = [col for col in datos.columns if col.startswith('against_')]
datos_entrenamiento = datos[columnas_against].values
# Guardar nombres y tipos para análisis posterior
nombres = datos['name'].values
tipos = datos[['type1', 'type2']].fillna('-').values

print("Forma del conjunto de datos de entrenamiento:", datos_entrenamiento.shape)

## SOM Setup


In [14]:
# Lista de tipos de Pokémon y mapeo de colores
TIPOS_POKEMON = ["normal", "fire", "water", "electric", "grass", "ice", "fighting", "poison", "ground",
                 "flying", "psychic", "bug", "rock", "ghost", "dragon", "dark", "steel", "fairy"]
TIPO_COLORES = {
    'normal': '#C6C6A7', 'fire': '#F5AC78', 'water': '#9DB7F5', 'electric': '#FAE078', 'grass': '#A7DB8D',
    'ice': '#BCE6E6', 'fighting': '#D67873', 'poison': '#C183C1', 'ground': '#EBD69D', 'flying': '#C6B7F5',
    'psychic': '#FA92B2', 'bug': '#C6D16E', 'rock': '#D1C17D', 'ghost': '#A292BC', 'dragon': '#A27DFA',
    'dark': '#A29288', 'steel': '#D1D1E0', 'fairy': '#F4BDC9'
}

def calcular_bmu(patron_de_entrada, matriz_de_pesos):
    """Encuentra la Unidad de Mejor Coincidencia (BMU) para un patrón de entrada."""
    distancias = np.linalg.norm(matriz_de_pesos - patron_de_entrada, axis=2)
    bmu_idx = np.unravel_index(np.argmin(distancias), distancias.shape)
    return matriz_de_pesos[bmu_idx], bmu_idx

def variacion_learning_rate(lr_inicial, i, n_iteraciones):
    """Calcula la tasa de aprendizaje decreciente."""
    return lr_inicial * (1 - i / n_iteraciones)

def variacion_vecindario(vecindario_inicial, i, n_iteraciones):
    """Calcula el tamaño del vecindario decreciente."""
    return vecindario_inicial * (1 - i / n_iteraciones)

def decay(distancia_BMU, vecindario_actual):
    """Calcula la influencia basada en la distancia a la BMU."""
    return np.exp(-distancia_BMU**2 / (2 * vecindario_actual**2))

def entrenar_som(datos, lado_mapa, periodo, learning_rate, normalizar_datos=True):
    """
    Entrena un SOM utilizando operaciones vectorizadas para mayor eficiencia.
    
    Parámetros:
    - datos: Array de forma (n_samples, n_features) con los datos de entrada.
    - lado_mapa: Tamaño del lado del mapa cuadrado.
    - periodo: Número de iteraciones de entrenamiento.
    - learning_rate: Tasa de aprendizaje inicial.
    - normalizar_datos: Si True, asume datos normalizados entre 0 y 1.
    
    Retorna:
    - matriz_pesos_inicial: Matriz de pesos antes del entrenamiento.
    - matriz_pesos: Matriz entrenada del SOM.
    """
    # Inicializa la matriz de pesos
    num_entradas = datos.shape[1]
    matriz_pesos = np.random.random((lado_mapa, lado_mapa, num_entradas))
    vecindario_inicial = lado_mapa // 2
    
    # Copia de la matriz inicial para análisis posterior
    matriz_pesos_inicial = matriz_pesos.copy()
    
    # Entrenamiento
    num_datos = datos.shape[0]
    for i in range(periodo):
        for idx in range(num_datos):
            patron = datos[idx]
            
            # Encuentra la BMU
            bmu, bmu_idx = calcular_bmu(patron, matriz_pesos)
            
            # Calcula parámetros actuales
            lr_actual = variacion_learning_rate(learning_rate, i, periodo)
            v_actual = variacion_vecindario(vecindario_inicial, i, periodo)
            
            # Calcula todas las distancias a la BMU
            x_coords = np.arange(matriz_pesos.shape[0])[:, None]
            y_coords = np.arange(matriz_pesos.shape[1])[None, :]
            dist_bmu = np.sqrt((x_coords - bmu_idx[0])**2 + (y_coords - bmu_idx[1])**2)

            # Inicializa matriz de influencia
            influencia = np.zeros_like(dist_bmu)
            
            # Función discreta: 1 para adyacentes (distancia = 1)
            influencia[dist_bmu == 1] = 1
            
            # Dentro del radio: usa función de decay
            mascara_radio = dist_bmu <= v_actual
            influencia[mascara_radio] = decay(dist_bmu[mascara_radio], v_actual)
            
            # Actualiza todos los pesos
            delta = lr_actual * influencia[:, :, None] * (patron - matriz_pesos)
            matriz_pesos += delta
            
        # Muestra progreso
        if (i + 1) % 100 == 0:
            print(f'Iteración {i + 1}/{periodo}')
    
    return matriz_pesos_inicial, matriz_pesos

In [15]:
def error_cuantificacion(datos, matriz_pesos):
    """Calcula el error de cuantificación promedio."""
    errores = [np.linalg.norm(patron - calcular_bmu(patron, matriz_pesos)[0]) for patron in datos]
    return np.mean(errores)

def error_topologico(datos, matriz_pesos):
    """Calcula el error topológico."""
    errores = 0
    for patron in datos:
        distancias = np.linalg.norm(matriz_pesos - patron, axis=2)
        bmu_idx = np.unravel_index(np.argmin(distancias), distancias.shape)
        distancias[bmu_idx] = np.inf
        segunda_bmu_idx = np.unravel_index(np.argmin(distancias), distancias.shape)
        distancia_bmus = np.linalg.norm(np.array(bmu_idx) - np.array(segunda_bmu_idx))
        if distancia_bmus > 1:
            errores += 1
    return errores / len(datos)

def contar_clases_activadas(datos, matriz_pesos):
    """Cuenta el número de neuronas activadas."""
    bmus = set()
    for patron in datos:
        _, bmu_idx = calcular_bmu(patron, matriz_pesos)
        bmus.add(bmu_idx)
    return len(bmus)

def visualizar_mapa_activaciones(matriz_activaciones, lado_mapa):
    """Visualiza el histograma 3D de activaciones."""
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    x, y = np.meshgrid(range(lado_mapa), range(lado_mapa))
    ax.bar3d(x.ravel(), y.ravel(), np.zeros_like(matriz_activaciones).ravel(), 
             1, 1, matriz_activaciones.ravel(), shade=True, color='blue')
    ax.set_title("Mapa de Activaciones (Histograma 3D)")
    plt.show()

def visualizar_mapa_distancias(matriz_pesos, lado_mapa):
    """Visualiza el mapa de distancias entre neuronas adyacentes."""
    distancias = np.zeros((lado_mapa, lado_mapa))
    for i in range(lado_mapa):
        for j in range(lado_mapa):
            vecinos = []
            for di, dj in [(0,1), (0,-1), (1,0), (-1,0)]:
                ni, nj = i + di, j + dj
                if 0 <= ni < lado_mapa and 0 <= nj < lado_mapa:
                    dist = np.linalg.norm(matriz_pesos[i, j] - matriz_pesos[ni, nj])
                    vecinos.append(dist)
            distancias[i, j] = np.mean(vecinos) if vecinos else 0
    sns.heatmap(distancias, cmap='viridis', square=True)
    plt.title("Matriz de Pesos Final")
    plt.show()

def visualizar_mapa_clasificacion(datos, matriz_pesos, nombres, tipos, lado_mapa):
    """Visualiza el mapa de clasificación con colores por tipo principal."""
    mapa_colores = np.full((lado_mapa, lado_mapa), '#FFFFFF')
    tipos_por_neurona = {}
    tipos_presentes = set()
    
    for i, patron in enumerate(datos):
        _, (x, y) = calcular_bmu(patron, matriz_pesos)
        pos = (x, y)
        tipo1 = tipos[i][0]
        tipo2 = tipos[i][1] if tipos[i][1] != '-' else None
        
        if pos not in tipos_por_neurona:
            tipos_por_neurona[pos] = set()
        
        tipos_por_neurona[pos].add(tipo1)
        tipos_presentes.add(tipo1)
        
        if tipo2:
            tipos_por_neurona[pos].add(tipo2)
            tipos_presentes.add(tipo2)
        
        # Asignar color basado en el tipo principal
        if mapa_colores[x, y] == '#FFFFFF':
            mapa_colores[x, y] = TIPO_COLORES.get(tipo1, '#FFFFFF')
    
    fig, ax = plt.subplots(figsize=(12, 10))
    
    for i in range(lado_mapa):
        for j in range(lado_mapa):
            color = mapa_colores[i, j]
            ax.add_patch(plt.Rectangle((j, lado_mapa-1-i), 1, 1, facecolor=color, edgecolor='black', linewidth=1))
            pos = (i, j)
            if pos in tipos_por_neurona:
                tipos_texto = '\n'.join(sorted(tipos_por_neurona[pos]))
                ax.text(j+0.5, lado_mapa-1-i+0.5, tipos_texto, ha='center', va='center',
                        color='white' if color != '#FFFFFF' else 'black', fontsize=7)
    
    ax.set_xlim(0, lado_mapa)
    ax.set_ylim(0, lado_mapa)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title("Clasificación de Pokémon en el SOM")
    
    legend_elements = [Patch(facecolor=TIPO_COLORES.get(tipo, '#FFFFFF'), edgecolor='black', label=tipo)
                       for tipo in sorted(tipos_presentes)]
    ax.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1, 0.5), title="Tipos de Pokémon")
    
    plt.show()

## SOM Entrenamiento

In [ ]:
# Rangos de parámetros a probar
lados_mapa = [20, 30, 40, 50]
periodos = [100, 100, 100, 100]
learning_rates = [0.01, 0.05, 0.1, 0.2]

resultados = []

for lm in lados_mapa:
    for p in periodos:
        for lr in learning_rates:
            print(f"Probando: lado_mapa={lm}, periodo={p}, learning_rate={lr}")
            # entrenar_som ahora devuelve matriz inicial y final
            _, matriz_pesos = entrenar_som(datos_entrenamiento, lm, p, lr)
            eq = error_cuantificacion(datos_entrenamiento, matriz_pesos)
            top = error_topologico(datos_entrenamiento, matriz_pesos)
            clases = contar_clases_activadas(datos_entrenamiento, matriz_pesos)
            resultados.append({'lado_mapa': lm, 'periodo': p, 'learning_rate': lr, 
                               'eq_error': eq, 'top_error': top, 'clases': clases})

# Convertir resultados a DataFrame
resultados_df = pd.DataFrame(resultados)
print(resultados_df)

# Encontrar los mejores parámetros (minimizar errores, maximizar clases)
best_result = resultados_df.loc[resultados_df['eq_error'].idxmin()]
print("Mejores parámetros basados en el error de cuantificación:")
print(best_result)

# Graficar errores
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
sns.scatterplot(data=resultados_df, x='lado_mapa', y='eq_error', hue='learning_rate', size='periodo')
plt.title('Error de Cuantificación')
plt.xlabel('Tamaño del Lado del Mapa')
plt.ylabel('Error de Cuantificación')
plt.subplot(1, 2, 2)
sns.scatterplot(data=resultados_df, x='lado_mapa', y='top_error', hue='learning_rate', size='periodo')
plt.title('Error Topológico')
plt.xlabel('Tamaño del Lado del Mapa')
plt.ylabel('Error Topológico')
plt.tight_layout()
plt.show()

In [ ]:
# Parámetros óptimos 
lado_mapa_opt = 50
periodo_opt = 500
learning_rate_opt = 0.1

# Entrenar el SOM
matriz_pesos_inicial, matriz_pesos_final = entrenar_som(datos_entrenamiento, lado_mapa_opt, periodo_opt, learning_rate_opt)

# Evaluar
n_clases = contar_clases_activadas(datos_entrenamiento, matriz_pesos_final)
eq_error = error_cuantificacion(datos_entrenamiento, matriz_pesos_final)
top_error = error_topologico(datos_entrenamiento, matriz_pesos_final)

# Calcular matriz de activaciones
matriz_activaciones = np.zeros((lado_mapa_opt, lado_mapa_opt))
for patron in datos_entrenamiento:
    _, bmu_idx = calcular_bmu(patron, matriz_pesos_final)
    matriz_activaciones[bmu_idx[0], bmu_idx[1]] += 1

# Mostrar resultados
print(f"Número de clases activadas: {n_clases}")
print(f"Error de Cuantificación: {eq_error:.4f}")
print(f"Error Topológico: {top_error:.4f}")

# Visualizar Matriz de Pesos Inicial 
distancias_inicial = np.zeros((lado_mapa_opt, lado_mapa_opt))
for i in range(lado_mapa_opt):
    for j in range(lado_mapa_opt):
        vecinos = []
        for di, dj in [(0,1), (0,-1), (1,0), (-1,0)]:
            ni, nj = i + di, j + dj
            if 0 <= ni < lado_mapa_opt and 0 <= nj < lado_mapa_opt:
                dist = np.linalg.norm(matriz_pesos_inicial[i, j] - matriz_pesos_inicial[ni, nj])
                vecinos.append(dist)
        distancias_inicial[i, j] = np.mean(vecinos) if vecinos else 0
sns.heatmap(distancias_inicial, cmap='viridis', square=True)
plt.title("Matriz de Pesos Inicial")
plt.show()

# Visualizar
visualizar_mapa_clasificacion(datos_entrenamiento, matriz_pesos_final, nombres, tipos, lado_mapa_opt)
visualizar_mapa_activaciones(matriz_activaciones, lado_mapa_opt)
visualizar_mapa_distancias(matriz_pesos_final, lado_mapa_opt)

In [ ]:
def visualizar_mapa_clasificacion_neuronas(datos, matriz_pesos, nombres, tipos, lado_mapa):
    """Visualiza el mapa de clasificación con regiones de colores por tipo."""
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Colores más vivos para las regiones
    colores_neuronas = {
        'electric': '#FFD700',    # Amarillo dorado
        'ice': '#00BFFF',        # Azul brillante
        'fire': '#FF4500',       # Rojo-naranja
        'water': '#0000FF',      # Azul puro
        'normal': '#A8A878',     # Normal
        'fighting': '#C03028',   # Lucha
        'flying': '#A890F0',     # Volador
        'poison': '#A040A0',     # Veneno
        'ground': '#E0C068',     # Tierra
        'rock': '#B8A038',       # Roca
        'bug': '#A8B820',        # Bicho
        'ghost': '#705898',      # Fantasma
        'steel': '#B8B8D0',      # Acero
        'grass': '#78C850',      # Planta
        'psychic': '#F85888',    # Psíquico
        'dragon': '#7038F8',     # Dragón
        'dark': '#705848',       # Siniestro
        'fairy': '#EE99AC'       # Hada
    }
    
    # Recolectar BMUs por tipo
    neuronas_por_tipo = {}
    tipos_presentes = set()
    
    for i, patron in enumerate(datos):
        _, (x, y) = calcular_bmu(patron, matriz_pesos)
        tipo1 = tipos[i][0]
        tipo2 = tipos[i][1] if tipos[i][1] != '-' else None
        
        if tipo1 not in neuronas_por_tipo:
            neuronas_por_tipo[tipo1] = []
        neuronas_por_tipo[tipo1].append((x, y))
        tipos_presentes.add(tipo1)
        
        if tipo2 and tipo2 != '-':
            if tipo2 not in neuronas_por_tipo:
                neuronas_por_tipo[tipo2] = []
            neuronas_por_tipo[tipo2].append((x, y))
            tipos_presentes.add(tipo2)
    
    # Para cada posición en el mapa, encontrar el tipo más cercano
    mapa_tipos = np.full((lado_mapa, lado_mapa), '', dtype=object)
    
    for i in range(lado_mapa):
        for j in range(lado_mapa):
            min_dist = float('inf')
            tipo_mas_cercano = None
            
            for tipo in tipos_presentes:
                if tipo in neuronas_por_tipo:
                    for x, y in neuronas_por_tipo[tipo]:
                        dist = np.sqrt((i-x)**2 + (j-y)**2)
                        if dist < min_dist:
                            min_dist = dist
                            tipo_mas_cercano = tipo
            
            mapa_tipos[i, j] = tipo_mas_cercano
    
    # Dibujar el mapa base con los colores
    for i in range(lado_mapa):
        for j in range(lado_mapa):
            tipo = mapa_tipos[i, j]
            color = colores_neuronas[tipo]
            ax.add_patch(Rectangle((j, lado_mapa-1-i), 1, 1, 
                                 facecolor=color,
                                 edgecolor='black',
                                 linewidth=0.5,
                                 alpha=0.6))
    
    ax.set_xlim(0, lado_mapa)
    ax.set_ylim(0, lado_mapa)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title("Clasificación de Pokémon en el SOM (Regiones)")
    
    # Crear leyenda
    legend_elements = [Patch(facecolor=colores_neuronas[tipo],
                           edgecolor='black', 
                           label=tipo)
                      for tipo in sorted(tipos_presentes)]
    ax.legend(handles=legend_elements, loc='center left', 
             bbox_to_anchor=(1, 0.5), title="Tipos de Pokémon")
    
    plt.tight_layout()
    plt.show()

# Usar la función con los datos de entrenamiento
visualizar_mapa_clasificacion_neuronas(datos_entrenamiento, matriz_pesos_final, nombres, tipos, lado_mapa_opt)

## SOM Clasificación

In [ ]:
# Cargar conjunto de datos de prueba
datos_prueba = pd.read_csv("pokemon_classify.csv")
nombres_prueba = datos_prueba['name'].values
tipos_prueba = datos_prueba[['type1', 'type2']].fillna('-').values
datos_prueba = datos_prueba[columnas_against].values

clasificaciones = {}
for i, patron in enumerate(datos_prueba):
    _, bmu_idx = calcular_bmu(patron, matriz_pesos_final)
    clasificaciones[nombres_prueba[i]] = bmu_idx
    
# Respuestas específicas
pikachu_bmu = clasificaciones.get('Pikachu')
articuno_bmu = clasificaciones.get('Articuno')
moltres_bmu = clasificaciones.get('Moltres')
slowbro_bmu = clasificaciones.get('Slowbro')
    
print(f"Pikachu se clasifica en: {pikachu_bmu}")
print(f"Articuno: {articuno_bmu}, Moltres: {moltres_bmu}")
print(f"Slowbro se clasifica en: {slowbro_bmu}")

## SOM Prueba

In [ ]:
def encontrar_neuronas_tipo(tipo, datos, matriz_pesos, tipos):
    """Encuentra todas las neuronas que representan un tipo dado."""
    bmus_tipo = []
    for i, patron in enumerate(datos):
        if tipos[i][0] == tipo or tipos[i][1] == tipo:
            _, bmu_idx = calcular_bmu(patron, matriz_pesos)
            bmus_tipo.append(bmu_idx)
    return list(set(bmus_tipo))  # Retorna lista de neuronas únicas para ese tipo

def visualizar_posicion_pokemons(pokemons, bmus_pokemons, neuronas_por_tipo, lado_mapa):
    """Visualiza las posiciones de múltiples Pokémon y todas las neuronas de sus tipos en el mismo mapa."""
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Dibujar el mapa base simplificado
    for i in range(lado_mapa):
        for j in range(lado_mapa):
            ax.add_patch(Rectangle((j, lado_mapa-1-i), 1, 1, facecolor='lightgray', edgecolor='black', linewidth=1))
    
    # Colores originales para los Pokémon
    colores_pokemons = {
        'Pikachu': '#FAE078',     # Color amarillo eléctrico
        'Articuno': '#BCE6E6',    # Color celeste hielo
        'Moltres': '#F5AC78',     # Color naranja fuego
        'Slowbro': '#9DB7F5'      # Color azul agua
    }
    
    # Colores más vivos para las neuronas de tipo
    colores_neuronas = {
        'electric': '#FFD700',    # Amarillo dorado
        'ice': '#00BFFF',        # Azul brillante
        'fire': '#FF4500',       # Rojo-naranja
        'water': '#0000FF'       # Azul puro
    }
    
    # Primero dibujamos todas las neuronas de tipo (para que queden en el fondo)
    tipos = ['electric', 'ice', 'fire', 'water']
    for tipo, pokemon in zip(tipos, pokemons):
        if tipo in neuronas_por_tipo:
            for neurona in neuronas_por_tipo[tipo]:
                x, y = neurona
                # Dibuja un rectángulo semitransparente para el área del tipo
                ax.add_patch(Rectangle((x, lado_mapa-1-y), 1, 1, 
                                     facecolor=colores_neuronas[tipo],
                                     alpha=0.3))
                # Dibuja un pequeño círculo para marcar el centro de la neurona
                ax.add_patch(Circle((x + 0.5, lado_mapa - 1 - y + 0.5), 0.15,
                                  facecolor=colores_neuronas[tipo],
                                  alpha=0.7))
    
    # Luego dibujamos los Pokémon (para que queden encima)
    for pokemon, bmu_pokemon, tipo in zip(pokemons, bmus_pokemons, tipos):
        # Posición del Pokémon
        if bmu_pokemon:
            x, y = bmu_pokemon
            ax.add_patch(Circle((x + 0.5, lado_mapa - 1 - y + 0.5), 0.3, 
                               color=colores_pokemons[pokemon], 
                               label=f'{pokemon} ({tipo})'))
            ax.text(x + 0.5, lado_mapa - 1 - y + 0.5, pokemon, 
                   ha='center', va='center', color='black', fontsize=8)
    
    ax.set_xlim(0, lado_mapa)
    ax.set_ylim(0, lado_mapa)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title("Posiciones de Pokémon y Neuronas de sus Tipos en el SOM")
    ax.legend(loc='upper right', bbox_to_anchor=(1.15, 1))
    plt.tight_layout()
    plt.show()

# Clasificar Pokémon de prueba y obtener BMUs
clasificaciones = {}
for i, patron in enumerate(datos_prueba):
    _, bmu_idx = calcular_bmu(patron, matriz_pesos_final)
    clasificaciones[nombres_prueba[i]] = bmu_idx

# Obtener BMUs para los Pokémon y todas las neuronas por tipo
pokemons = ['Pikachu', 'Articuno', 'Moltres', 'Slowbro']
bmus_pokemons = [clasificaciones.get(pokemon) for pokemon in pokemons]

# Crear diccionario de neuronas por tipo
neuronas_por_tipo = {
    'electric': encontrar_neuronas_tipo('electric', datos_entrenamiento, matriz_pesos_final, tipos),
    'ice': encontrar_neuronas_tipo('ice', datos_entrenamiento, matriz_pesos_final, tipos),
    'fire': encontrar_neuronas_tipo('fire', datos_entrenamiento, matriz_pesos_final, tipos),
    'water': encontrar_neuronas_tipo('water', datos_entrenamiento, matriz_pesos_final, tipos)
}

# Visualizar todas las posiciones en un solo mapa
visualizar_posicion_pokemons(pokemons, bmus_pokemons, neuronas_por_tipo, lado_mapa_opt)